In [3]:
import numpy as np
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import algorithm_globals, QuantumInstance
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
import pandas as pd
from qiskit_machine_learning.algorithms import PegasosQSVC, QSVC
from qiskit_machine_learning.kernels import QuantumKernel

algorithm_globals.random_seed = 10598

In [4]:
# number of qubits is equal to the number of features
num_qubits = 4

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [5]:
pegasos_backend = QuantumInstance(
    Aer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

features, labels = make_blobs(n_samples=20, n_features = 4, centers = 2, random_state = 3, shuffle = True)
features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, train_size=15, shuffle=False)

feature_map = ZZFeatureMap(feature_dimension = num_qubits, reps=1)
qkernel = QuantumKernel(feature_map = feature_map, quantum_instance=pegasos_backend)

pegasos_qsvc = PegasosQSVC(quantum_kernel = qkernel, C = C, num_steps = tau)
pegasos_qsvc.seed = 7

# training
pegasos_qsvc.fit(train_features, train_labels)

# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"PegasosQSVC classification test score: {pegasos_score}")

PegasosQSVC classification test score: 0.8


In [6]:
qsvc = QSVC(quantum_kernel = qkernel)

# training
qsvc.fit(train_features, train_labels)

# testing
qsvc = qsvc.score(test_features, test_labels)
print(f"QSVC classification test score: {qsvc}")

QSVC classification test score: 0.8


In [1]:
import qiskit_machine_learning
qiskit_machine_learning.__version__

'0.4.0'